In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.utils.random_sequence import (powerlaw_sequence)
from tqdm import tqdm
import sys
import copy
from numba import jit
from multiprocessing import Pool

from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='jshtml')
# import networkx.utils.create_degree_sequence as create_degree_sequence
%matplotlib notebook

## Build a network

In [ ]:
def create_degree_sequence(n, sfunction=None, max_tries=50, **kwds):
    """ Attempt to create a valid degree sequence of length n using
    specified function sfunction(n,**kwds).

    Parameters
    ----------
    n : int
        Length of degree sequence = number of nodes
    sfunction: function
        Function which returns a list of n real or integer values.
        Called as "sfunction(n,**kwds)".
    max_tries: int
        Max number of attempts at creating valid degree sequence.

    Notes
    -----

    Repeatedly create a degree sequence by calling sfunction(n,**kwds)
    until achieving a valid degree sequence. If unsuccessful after
    max_tries attempts, raise an exception.
    
    For examples of sfunctions that return sequences of random numbers,
    see networkx.Utils.

    Examples
    --------
    >>> from networkx.utils import uniform_sequence, create_degree_sequence
    >>> seq=create_degree_sequence(10,uniform_sequence)
    """
    tries=0
    max_deg=n
    while tries < max_tries:
        trialseq=sfunction(n,**kwds)
        # round to integer values in the range [0,max_deg]
        seq=[min(max_deg, max( int(round(s)),0 )) for s in trialseq]
        # if graphical return, else throw away and try again
        if nx.is_graphical(seq):
            return seq
        tries+=1
    raise nx.NetworkXError("Exceeded max (%d) attempts at a valid sequence."%max_tries)

In [ ]:
#### code from http://nbviewer.jupyter.org/gist/Midnighter/248f1a5d8c21b39525ae

exp = 2.4
seed = 123456789
num = 1000
print("exponent = -{0:.2f}".format(exp))
sequence = create_degree_sequence(num, powerlaw_sequence, exponent=exp)
graph = nx.configuration_model(sequence, seed=seed)
# count parallel edges and avoid counting A-B as well as B-A
num_par = sum(
    len(graph[node][neigh]) for node in graph
    for neigh in graph.neighbors(node)) // 2
print("graph has {0:d} parallel edges".format(num_par))
loops = graph.selfloop_edges()
# remove them
graph = nx.Graph(graph)
graph.remove_edges_from(loops)
# get largest connected component
# unfortunately, the iterator over the components is not guaranteed to be sorted by size
components = sorted(nx.connected_components(graph), key=len, reverse=True)
lcc = graph.subgraph(components[0])
print("size of largest connected component = {0:d}".format(len(lcc)))
# new degree sequence
simple_seq = [deg for (node, deg) in lcc.degree()]
# create histograms
counts = np.bincount(sequence)
mask = (counts > 0)
plt.figure()
plt.plot(
    np.arange(len(counts))[mask],
    counts[mask] / counts.sum(),
    "o",
    label="MultiGraph")
simple_counts = np.bincount(simple_seq)
mask = (simple_counts > 0)
# distribution is shifted for visibility
plt.plot(
    np.arange(len(simple_counts))[mask],
    simple_counts[mask] / simple_counts.sum() / 10.0,
    "o",
    label="Simple LCC")
x = np.arange(1, len(counts))
plt.plot(x, np.power(x, -exp))
plt.xlabel(r"Degree $k$")
plt.xscale("log")
plt.ylabel(r"Probability $P(k)$")
plt.yscale("log")
plt.title(r"$N = {0:d}, \\quad \\lambda = {1:.2f}$".format(num, exp))
plt.legend(loc="best")
plt.show()
nx.powerlaw_cluster_graph

In [ ]:
plt.figure()
pos = nx.spring_layout(graph)
nx.draw(graph, pos, node_color='b', node_size=10, with_labels=False)
plt.show()

In [ ]:
p = 0.1
num = 100
graph = nx.fast_gnp_random_graph(num, p, seed=seed, directed=False)

plt.figure()
pos = nx.spring_layout(graph)
nx.draw(graph, pos, node_color='b', node_size=5, with_labels=False)
plt.show()

plt.figure()
degree_sequence = sorted([d for n, d in graph.degree()], reverse=True)
# print "Degree sequence", degree_sequence
dmax = max(degree_sequence)

plt.semilogy(degree_sequence, 'b-', marker='o')
plt.title("Degree rank plot")
plt.ylabel("degree")
plt.xlabel("rank")
plt.show()

## Implementation of disease spreading

In [ ]:
class Person():
    """ 
    Contains data about status of infection for each person.
    
    time_since_infection equals -1 if person is not infected.
    
    The disease status is 0 for no disease, 1 for the sensitive
    strain and 2 for the resistant strain.
    """
    
    def __init__(self,i):
        self.identifier = i
        self.disease_status = 0
        self.time_since_infection = -1

In [ ]:
# @jit()
def model(beta, tau, nu, init, num_steps ,graph, doInit = False):
    num_infected  = np.zeros(num_steps)
    num_res = np.zeros(num_steps)

    infected = set()
    
    if doInit:
        for i in range(len(graph)):
            graph.node[i]["Data"].disease_status = 0

        
        for i in init:
            graph.node[i]["Data"].disease_status = 1
            infected.add(i)
    else:
        for i in range(len(graph)):
            if graph.node[i]["Data"].disease_status:
                infected.add(i)
      
    
    # iterate over time
    for t in tqdm(range(num_steps), position = 0):
        infected_copy = infected.copy()

        # iterate over infecteds 
        for i in infected_copy:

            # prob of recovering
            if np.random.rand() < nu:
                graph.node[i]["Data"].disease_status = 0
                infected.remove(i)

            # prob of treatment
            elif graph.node[i]["Data"].disease_status == 1 and np.random.rand() < tau*(1-mu):
                graph.node[i]["Data"].disease_status = 0
                infected.remove(i)

            # prob of getting the resistant strain
            elif graph.node[i]["Data"].disease_status == 1 and np.random.rand() < tau*mu:
                graph.node[i]["Data"].disease_status = 2

            # spreading of disease to neigbours
            if graph.node[i]["Data"].disease_status:
                for neighbor in graph.neighbors(i):
                    if graph.node[neighbor]["Data"].disease_status == 0 and np.random.rand() < beta:
                        highest_disease = max(
                            graph.node[i]["Data"].disease_status,
                            graph.node[neighbor]["Data"].disease_status)
                        graph.node[i]["Data"].disease_status = highest_disease
                        graph.node[neighbor]["Data"].disease_status = highest_disease
                        infected.add(i)
                        infected.add(neighbor)

        # keep track of total number of resistant infecteds
        tot = 0
        for i in infected:
            if graph.node[i]["Data"].disease_status == 2:
                tot += 1
        num_res[t] = tot 
        num_infected[t] = len(infected)
        
    return (num_res, num_infected)

In [ ]:
# set person class to each node
for i in range(len(graph)):
    graph.node[i]["Data"] = Person(i)

# create set of infecteds, initialize with 10 infecteds

init = np.random.choice(len(graph), 10, replace=False)

t = 0
t_tot = 20
time_step = 365
dt = 1 / (time_step)
D = 0.19 * time_step
mu = 1e-3

# constants of model
PHI = [0.65, 0.50]
BETA = [0.59, 0.87]

# keep track of infecteds and steps
num_steps = int(t_tot / dt)
num_infected = np.zeros((2, num_steps))
num_res = np.zeros((2, num_steps))

for sextype in range(2):
    phi = PHI[sextype]
    beta = BETA[sextype]

    beta = 1 - (1 - beta)**(1 / time_step)
    nu = 2e-3 * (1 - phi)
    tau = 8e-3 * phi

    
    num_res[sextype,:], num_infected[sextype,:]= model(beta, tau, nu, init, num_steps, copy.deepcopy(graph), True)

## Plotting

In [ ]:
# plot number of infecteds
t = np.linspace(0,t_tot, num_steps)    
plt.figure()
print(np.mean(num_infected))
plt.plot(t, num_infected[0,:], label='all infecteds MSM')
plt.plot(t, num_res[0,:], label='resistant infecteds MSM')
plt.plot(t, num_infected[1,:], label='all infecteds HMW')
plt.plot(t, num_res[1,:], label='resistant infecteds HMW')
plt.legend()
plt.show()

plt.figure()
plt.plot(t,num_res[0,:]/num_infected[0,:], label='fraction of resistant strain MSM')
plt.plot(t,num_res[1,:]/num_infected[1,:], label='fraction of resistant strain HMW')
# plt.plot(num_res)
plt.legend()
plt.show()

In [ ]:
# set person class to each node
for i in range(len(graph)):
    graph.node[i]["Data"] = Person(i)

# create set of infecteds, initialize with 10 infecteds

init = np.random.choice(len(graph), 10, replace=False)

t = 0
t_tot = 10
time_step = 365
dt = 1 / (time_step)
D = 0.19 * time_step
mu = 1e-3

# constants of model
PHI = [0.65, 0.50]
BETA = np.linspace(0.15, 0.3, 10)

# keep track of infecteds and steps
num_steps = int(t_tot / dt)
num_infected = np.zeros((len(BETA), num_steps))
num_res = np.zeros((len(BETA), num_steps))

for i, beta in enumerate(BETA):
    phi = PHI[0]

    BETA = 1 - (1 - BETA)**(1 / time_step)
    nu = 2e-3 * (1 - phi)
    tau = 8e-3 * phi
    num_res[i,:], num_infected[i,:]= model(beta, tau, nu, init, num_steps, copy.deepcopy(graph))


# with multiprocessing.Pool(processes=3) as pool:
#     results = pool.starmap(merge_names, product(names, repeat=2))

In [ ]:
t = np.linspace(0,t_tot, num_steps)    
plt.figure()

plt.plot(BETA, num_infected[:,-1], label='all infecteds %i' % BETA[i])
plt.plot(BETA, num_res[:,-1], label='resistant infecteds %i'% BETA[i])
# plt.legend()
plt.show()

# plt.figure()
# plt.plot(t,num_res[0,:]/num_infected[0,:], label='fraction of resistant strain MSM')
# plt.plot(t,num_res[1,:]/num_infected[1,:], label='fraction of resistant strain HMW')
# # plt.plot(num_res)
# plt.legend()
# plt.show()

In [ ]:
# set person class to each node
for i in range(len(graph)):
    graph.node[i]["Data"] = Person(i)

# create set of infecteds, initialize with 10 infecteds

init = np.random.choice(len(graph), 10, replace=False)

t = 0
t_tot = 20
time_step = 365
dt = 1 / (time_step)
D = 0.19 * time_step
mu = 1e-3

# constants of model
PHI = [0.65, 0.50]
BETA = [0.59, 0.87]

# keep track of infecteds and steps
# num_steps = int(t_tot / dt)

# for sextype in range(2):
sextype = 0

phi = PHI[sextype]
beta = BETA[sextype]

beta = 1 - (1 - beta)**(1 / time_step)
nu = 2e-3 * (1 - phi)
tau = 8e-3 * phi


model(beta, tau, nu, init, 0, graph, doInit=True)

# get values of disease status
values = []
for j in range(len(graph.nodes())):
    if graph.node[j]["Data"].disease_status == 0:
        values.append("Black")
    elif graph.node[j]["Data"].disease_status == 1:
        values.append("Blue")
    elif graph.node[j]["Data"].disease_status == 2:
        values.append("Red")

# for node in graph.nodes():
#     print(node)

# plt.figure()
# pos = nx.spring_layout(graph)
# nx.draw(graph_total[2], pos, node_color=values, node_size=50, with_labels=True)
# plt.show()


# make figure to plot in
fig = plt.figure(figsize = (10,5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)


im1, = ax2.plot([], [], c = 'b', label='all infecteds MSM')

im2, = ax2.plot([], [], c = 'r', label='resistant infecteds MSM')
ax2.legend()
ax2.set_xlim(0,20)
ax2.set_ylim(0,1000)
# plot initial conditions (number of infecteds)
# cv0 = xyz_values[0,:,:,1]
pos = nx.spring_layout(graph) 
plt.close()
# function to animate each timestep
ani_step = 183
frames = 40
num_infected = np.zeros(ani_step*frames)
num_res = np.zeros(ani_step*frames)



def update(i):
    temp_res, temp_infected = model(beta, tau, nu, init, ani_step, graph, doInit=False)
    num_infected[i*ani_step:(i+1)*ani_step] = temp_infected
    num_res[i*ani_step:(i+1)*ani_step] = temp_res
    t = np.linspace(0,i*ani_step/365, i*ani_step) 
    im1.set_data(t, num_infected[:i*ani_step])
    im2.set_data(t, num_res[:i*ani_step])
    ax1.clear()
    print(i)
    
    # solution at step i
    values = []
    for j in range(len(graph.nodes())):
        if graph.node[j]["Data"].disease_status == 0:
            values.append("Black")
        elif graph.node[j]["Data"].disease_status == 1:
            values.append("Blue")
        elif graph.node[j]["Data"].disease_status == 2:
            values.append("Red")

    # for node in graph.nodes():
    #     print(node)
    nx.draw(graph, pos=pos, node_color=values, node_size=10, ax=ax1, clim = 2)
    
    # Scale plot ax
    ax1.set_title("Frame %d:    "%(i+1), fontweight="bold")
    ax1.set_xticks([])
    ax1.set_yticks([])
    
# animate!
animation.FuncAnimation(fig, update, frames=40, interval=100, repeat=True)

In [ ]:
num_infected[0,:].shape